In [83]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import optuna
from scipy.integrate import cumtrapz
import warnings
warnings.filterwarnings(action='ignore')


#### Data loading

In [84]:
train=pd.read_csv('./train_features.csv')
train_labels=pd.read_csv('./train_labels.csv')
test=pd.read_csv('./test_features.csv')

submission=pd.read_csv('./sample_submission.csv')

pd.options.display.max_columns=50

In [85]:
train.shape

(1875000, 8)

#### Feature engneering

In [86]:
x_train = []

for uid in tqdm(train['id'].unique()):
    temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
    x_train.append(temp)

x_train = np.array(x_train, np.float32)
x_train = x_train[:,:,:,np.newaxis]

x_test = []

for uid in tqdm(test['id'].unique()):
    temp = np.array(test[test['id'] == uid].iloc[:,2:], np.float32).T
    x_test.append(temp)

x_test = np.array(x_test, np.float32)
x_test = x_test[:,:,:,np.newaxis]

100%|██████████| 782/782 [00:00<00:00, 963.49it/s]


In [87]:
x_train.shape

(3125, 6, 600, 1)

In [88]:
X=x_train.reshape(3125, 600, -1)

In [89]:
X.shape

(3125, 600, 6)

In [90]:
X

array([[[ 1.20608664e+00,  1.28769648e+00,  1.30460918e+00,
          1.29309487e+00,  1.30088687e+00,  1.28930390e+00],
        [ 1.28140545e+00,  1.24227333e+00,  1.19887137e+00,
          1.11367714e+00,  1.19224143e+00,  1.23009431e+00],
        [ 1.17520964e+00,  1.17025077e+00,  1.07527697e+00,
          9.93865490e-01,  9.56588447e-01,  9.77357328e-01],
        ...,
        [ 1.99831734e+01,  2.08915806e+01,  1.97046967e+01,
          2.17813911e+01,  1.78339272e+01,  1.67689190e+01],
        [ 1.56782789e+01,  1.63014698e+01,  1.22182951e+01,
          1.21180601e+01,  1.52977333e+01,  1.54031487e+01],
        [ 1.61577053e+01,  2.29730167e+01,  2.31745968e+01,
          1.89320698e+01,  2.06315765e+01,  3.21230087e+01]],

       [[-2.11794689e-01, -2.56963193e-01, -2.57004678e-01,
         -1.89641625e-01, -1.62570298e-01, -1.63815469e-01],
        [-1.58589363e-01, -1.45079434e-01, -1.61238298e-01,
         -1.65159091e-01, -1.79104164e-01, -2.10551843e-01],
        [-2.25943

#####  가속도, 자이로, (자이로-가속도) 센서값을 에너지로 표현

In [24]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [27]:
train.shape

(1875000, 11)

100%|██████████| 782/782 [00:00<00:00, 978.99it/s]


(3125, 9, 600, 1)

(3125, 600, 9)

###### id별 데이터는 0.02초마다 측정된 값들이기 때문에 이전 시간 대비 변화량 적용

In [ ]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [ ]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

In [ ]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

In [ ]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [ ]:
train=pd.concat(train_dt)

In [ ]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

In [ ]:
test=pd.concat(test_dt)

In [5]:
# feature_names = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']
# #,'acc_Energy','gy_Energy','gy_acc_Energy'
# grad_cols=[]
# for col in feature_names:
#     grad_cols.append(f"grad_{col}")

# integ_cols = []
# for col in feature_names:
#     integ_cols.append(f"integ_{col}")
    

    
# total_feature_names = feature_names + grad_cols + integ_cols 

In [6]:
# # 기울기(gradient) 구하기 범용 함수 : np.gradient()
# # 

# for uid in tqdm(train['id'].unique()):
#     temp = train.loc[train['id']==uid, feature_names]
#     grad = np.gradient(temp, axis=0)
#     train.loc[train['id']==uid, grad_cols] = grad
    
# for uid in tqdm(test['id'].unique()):
#     temp = test.loc[test['id']==uid, feature_names]
#     grad = np.gradient(temp, axis=0)
#     test.loc[test['id']==uid, grad_cols] = grad
    
# print(train.shape, test.shape)
# train.head()

100%|██████████| 782/782 [00:09<00:00, 80.34it/s]

(1875000, 14) (469200, 14)


,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,grad_acc_x,grad_acc_y,grad_acc_z,grad_gy_x,grad_gy_y,grad_gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,0.081610,-0.019602,-0.033997,0.894708,-8.590094,6.748896
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,0.049261,-0.007872,-0.052468,-1.512835,-6.786777,3.328242
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,0.002699,-0.015696,-0.016383,1.204943,-7.229370,-1.263398
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,-0.001861,0.003679,0.015430,3.951992,-6.891998,-1.470803
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,-0.001896,0.007851,-0.017338,3.676439,-5.326397,-0.916218


In [7]:
# for uid in tqdm(train['id'].unique()):
#     temp = train.loc[train['id']==uid, feature_names]
#     integ = cumtrapz(temp, initial=0, axis=0)
#     train.loc[train['id']==uid, integ_cols] = integ
    
# for uid in tqdm(test['id'].unique()):
#     temp = test.loc[test['id']==uid, feature_names]
#     integ = cumtrapz(temp, initial=0, axis=0)
#     test.loc[test['id']==uid, integ_cols] = integ
    
# print(train.shape, test.shape)
# train.head()

100%|██████████| 782/782 [00:09<00:00, 78.97it/s]

(1875000, 20) (469200, 20)


,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,grad_acc_x,grad_acc_y,grad_acc_z,grad_gy_x,grad_gy_y,grad_gy_z,integ_acc_x,integ_acc_y,integ_acc_z,integ_gy_x,integ_gy_y,integ_gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,0.081610,-0.019602,-0.033997,0.894708,-8.590094,6.748896,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,0.049261,-0.007872,-0.052468,-1.512835,-6.786777,3.328242,1.246892,-0.189173,-0.165445,-0.144254,-34.844056,-28.301664
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,0.002699,-0.015696,-0.016383,1.204943,-7.229370,-1.263398,2.543044,-0.386217,-0.383358,-1.801342,-76.474890,-53.275086
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,-0.001861,0.003679,0.015430,3.951992,-6.891998,-1.470803,3.841896,-0.598957,-0.617655,-2.253488,-125.335094,-79.511907
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,-0.001896,0.007851,-0.017338,3.676439,-5.326397,-0.916218,5.138887,-0.808019,-0.836521,1.246358,-181.087295,-107.219530


In [9]:
# from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler


# scaler = StandardScaler()

# Xtrain_scaled = pd.DataFrame(data=scaler.fit_transform(train[total_feature_names]), index=train.index, columns=total_feature_names)
# Xtest_scaled = pd.DataFrame(data=scaler.transform(test[total_feature_names]), index=test.index, columns=total_feature_names)

# print(Xtrain_scaled.shape, Xtest_scaled.shape)

(1875000, 18) (469200, 18)


In [12]:
# X = np.array(Xtrain_scaled[total_feature_names]).reshape(-1, 600, len(total_feature_names)).astype('float32')
# Xtest= np.array(Xtest_scaled[total_feature_names]).reshape(-1, 600, len(total_feature_names)).astype('float32')


# y = train_labels['label'].values
# y = tf.keras.utils.to_categorical(train_labels['label']) 
# y.shape

# print(X.shape, y.shape, Xtest.shape, submission.shape)

(3125, 600, 18) (3125, 61) (782, 600, 18) (782, 62)


##### Standard scaling 적용


In [ ]:
# col=train.columns
# train_s=train.copy()
# test_s=test.copy()

In [ ]:
# from sklearn.preprocessing import StandardScaler,MinMaxScaler

# scaler = StandardScaler()

# train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
# train_sc = pd.DataFrame(data = train_s,columns =col)

# test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
# test_sc = pd.DataFrame(data = test_s,columns =col)

# X=np.array(train_sc.iloc[:,2:]).reshape(3125, 600, -1)
# X.shape

# test_x=np.array(test_sc.iloc[:,2:]).reshape(782, 600, -1)
# test_x.shape

# y = train_labels['label'].values
# y = tf.keras.utils.to_categorical(train_labels['label']) 
# y.shape

In [32]:
test_x=x_test.reshape(782, 600, -1)
test_x.shape

y = train_labels['label'].values
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

##### 모델링

+ CNN, LSTM, CNN+LSTM 등 여러 구조 적용해보다가 CNN에서 Flatten 없이 Global average pooling 한 구조가 가장 성능이 좋아 채택했습니다.


In [33]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from keras.layers import Dense, LSTM,Bidirectional,Dropout
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling1D
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [34]:
# X=x_train.reshape(3125, 600, -1)
# X.shape

In [35]:
# test_x=x_test.reshape(782, 600, -1)
# test_x.shape

##### 모델 구조 

In [79]:

model= keras.Sequential()
model.add(Conv1D(filters=128, kernel_size=9, kernel_initializer='he_uniform' ,padding="same",input_shape=X.shape[1:], activation="relu"))
model.add(MaxPooling1D())
model.add(Conv1D(filters=256, kernel_size=9, padding="same",input_shape=X.shape[1:], activation="relu"))

model.add(Conv1D(filters=128, kernel_size=9, padding="same",input_shape=X.shape[1:], activation="relu"))
model.add(MaxPooling1D())
model.add(Conv1D(filters=64, kernel_size=9, padding="same",input_shape=X.shape[1:], activation="relu"))
model.add(GlobalAveragePooling1D())

#model.add(Flatten())

model.add(Dense(128, activation="relu"))
model.add(Dropout(rate=0.1))
model.add(Dense(64, activation="relu"))
model.add(Dropout(rate=0.2))


#model.add(Dense(64, activation="relu"))

model.add(Dense(61, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics='accuracy')
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_55 (Conv1D)           (None, 600, 128)          10496     
_________________________________________________________________
max_pooling1d_39 (MaxPooling (None, 300, 128)          0         
_________________________________________________________________
conv1d_56 (Conv1D)           (None, 300, 256)          295168    
_________________________________________________________________
conv1d_57 (Conv1D)           (None, 300, 128)          295040    
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 150, 128)          0         
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 150, 64)           73792     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 64)              

In [80]:

# seed(2021)
# tf.random.set_seed(2021)

# input_layer = keras.layers.Input(X.shape[1:])
# conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
# conv1 = keras.layers.BatchNormalization()(conv1)
# conv1 = keras.layers.Activation(activation='relu')(conv1)
# conv1 = keras.layers.Dropout(rate=0.3)(conv1)

# conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
# conv2 = keras.layers.BatchNormalization()(conv2)
# conv2 = keras.layers.Activation('relu')(conv2)
# conv2 = keras.layers.Dropout(rate=0.4)(conv2)

# conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
# conv3 = keras.layers.BatchNormalization()(conv3)
# conv3 = keras.layers.Activation('relu')(conv3)
# conv3 = keras.layers.Dropout(rate=0.5)(conv3)

# gap = keras.layers.GlobalAveragePooling1D()(conv3)

# output_layer = keras.layers.Dense(61, activation='softmax')(gap)

# model = keras.models.Model(inputs=input_layer, outputs=output_layer)

# model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics='accuracy')
# model.summary()

##### 10-fold StratifiedKFold

In [81]:
skf = StratifiedKFold(n_splits = 10, random_state = 123, shuffle = True)
reLR = ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
es =EarlyStopping(monitor='val_loss', patience=10, mode='min')



history = model.fit(X, y, epochs = 70, verbose=1, batch_size=64, validation_split = 0.2, callbacks=[es,reLR])



Epoch 1/70
40/40 [==============================] - 3s 52ms/step - loss: 4.9023 - accuracy: 0.3780 - val_loss: 2.8727 - val_accuracy: 0.4976
Epoch 2/70
40/40 [==============================] - 2s 38ms/step - loss: 2.6965 - accuracy: 0.4828 - val_loss: 2.4430 - val_accuracy: 0.4976
Epoch 3/70
40/40 [==============================] - 2s 39ms/step - loss: 2.5510 - accuracy: 0.4840 - val_loss: 2.4192 - val_accuracy: 0.4976
Epoch 4/70
40/40 [==============================] - 2s 38ms/step - loss: 2.4230 - accuracy: 0.4904 - val_loss: 2.2612 - val_accuracy: 0.5168
Epoch 5/70
40/40 [==============================] - 2s 38ms/step - loss: 2.3353 - accuracy: 0.4936 - val_loss: 2.1588 - val_accuracy: 0.5168
Epoch 6/70
40/40 [==============================] - 2s 39ms/step - loss: 2.2445 - accuracy: 0.4964 - val_loss: 2.3379 - val_accuracy: 0.5136
Epoch 7/70
40/40 [==============================] - 2s 38ms/step - loss: 2.3034 - accuracy: 0.5012 - val_loss: 2.1449 - val_accuracy: 0.5248
Epoch 8/70
40

##### 성능 확인 및 제출

In [82]:
# 8. 모델 평가하기
model.evaluate(X, y)

98/98 [==============================] - 1s 8ms/step - loss: 0.6877 - accuracy: 0.8253


[0.6877084970474243, 0.8252800107002258]

In [ ]:
pred = model.predict(test_x)

pred

In [ ]:
# submission=pd.read_csv('./data/sample_submission.csv')
# submission.iloc[:,1:]=pred
# submission

In [ ]:
# submission.to_csv('sub_kfold_stratified_10_adam_fft_0.5.csv',index=False)